Get CAI weights for organisms with no expression data. How?
We will get all the CDSs of their genes, calculate the effective number of codons (ENC) and get the top biased 5% of genes.
This will be an estimation of the highly expressed genes. We will calculate the CAI weights of these organisms.
The CAI weights for all organisms are saved in "../Results/codon_usage_tables/". 

*This notebook was ran on my computer, not power*


In [ ]:
from Bio import SeqIO
import pandas as pd
import numpy as np
from Bio.Seq import Seq
from CAI import CAI, relative_adaptiveness
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import Bio
import pickle
from multiprocessing import  Pool
from pandarallel import pandarallel
from tqdm import tqdm
import os
from Utils_Tal import SynonymousCodons


In [3]:
''' This function returns "True" if there are only the "ACGT" alphabet in the sequence and False otherwise '''
def find_ambigious_alphabet(Myseq:str) -> bool: #find any sequence were letters other than A,C,G or T appear. 
    good = "ACGT"
    return(all(c in good for c in Myseq))

In [4]:
''' This function calculates the ENC score for a single sequence '''
def calc_ENC(MySeq:str) -> float:
       
    #initialize codon count:
    codons_count = {
      "TTT": 0, "TTC": 0, "TTA": 0, "TTG": 0, "CTT": 0,
      "CTC": 0, "CTA": 0, "CTG": 0, "ATT": 0, "ATC": 0,
      "ATA": 0, "ATG": 0, "GTT": 0, "GTC": 0, "GTA": 0,
      "GTG": 0, "TAT": 0, "TAC": 0, "TAA": 0, "TAG": 0,
      "CAT": 0, "CAC": 0, "CAA": 0, "CAG": 0, "AAT": 0,
      "AAC": 0, "AAA": 0, "AAG": 0, "GAT": 0, "GAC": 0,
      "GAA": 0, "GAG": 0, "TCT": 0, "TCC": 0, "TCA": 0,
      "TCG": 0, "CCT": 0, "CCC": 0, "CCA": 0, "CCG": 0,
      "ACT": 0, "ACC": 0, "ACA": 0, "ACG": 0, "GCT": 0,
      "GCC": 0, "GCA": 0, "GCG": 0, "TGT": 0, "TGC": 0,
      "TGA": 0, "TGG": 0, "CGT": 0, "CGC": 0, "CGA": 0,
      "CGG": 0, "AGT": 0, "AGC": 0, "AGA": 0, "AGG": 0,
      "GGT": 0, "GGC": 0, "GGA": 0, "GGG": 0}

    #count codons in the sequence
    for i in range(0, len(MySeq), 3):
        codon = MySeq[i:i + 3]
        if codon in codons_count:
            codons_count[codon] += 1
        else:
            raise TypeError("illegal codon:",codon,"in gene")

    #count the AAs in the sequence
    aa_seq =  str(Seq.translate(MySeq))
    X = ProteinAnalysis(aa_seq)
    aa_count = X.count_amino_acids() #ignores the stop codons, consider this later

    # How are we going to calculate the ENC?
    #1. Calculate Faa for each amino acid
    #2. Calculate the mean Faa for amino acids with the same amount of synonymous codons (F_lenx)
    #3. Use these results in the final formula

    Faa_all = pd.DataFrame() # every column will be an AA, every row would be a number of synonymous codons. we will save the Faa scores of each 
    #AA in the appropriate row, according to the number of the synonymous codons coding for it. Then, we can average each row to obtain F_lenx for each length. 

    for AA in aa_count.keys():
        cur_codons_squared_frequency = [] #we want to calculate the sqaured frequency of each synonymous codon of this current AA
        cur_AA_num_appearances = aa_count[AA]
        if cur_AA_num_appearances != 0: #meaning, if this AA is actually in the gene
            num_of_syn_codons = len(SynonymousCodons[AA]) #get the number of synonymous codons for this AA
            for codon in SynonymousCodons[AA]:
                cur_codon_squared_frequency = (codons_count[codon]/cur_AA_num_appearances)**2
                cur_codons_squared_frequency.append(cur_codon_squared_frequency)

            cur_Faa = np.sum(cur_codons_squared_frequency) #the Faa score for the current AA
            Faa_all.loc[num_of_syn_codons,AA] = cur_Faa # assign the Faa score of the current AA to the right position in the Faa_all dataframe

    # get F_lenx for each number of synonymous codons. 
    Faa_per_codon_amount = Faa_all.mean(axis = 1) #averaging the Faa for AAs with same amount of synonymous codons

    #final calculation using the known formula:
    
    if (4 not in Faa_per_codon_amount.index.to_list()) | (2 not in Faa_per_codon_amount.index.to_list()) | (6 not in Faa_per_codon_amount.index.to_list()) : #(either F2, F4 or F6 doesnt exist)
        ENC = np.nan #dont consider this gene at all 
        
    else:
        if 3 not in Faa_per_codon_amount.index.to_list() : # only F3 doesnt exist
            ENC = 2 + (9/Faa_per_codon_amount.loc[2]) + (1/(Faa_per_codon_amount[2]+Faa_per_codon_amount[4])/2) + (5/Faa_per_codon_amount[4]) + (3/Faa_per_codon_amount[6])

        else: # all exist
            ENC = 2 + (9/Faa_per_codon_amount.loc[2]) + (1/Faa_per_codon_amount[3]) + (5/Faa_per_codon_amount[4]) + (3/Faa_per_codon_amount[6])  
        
    return(ENC)

In [4]:
# iterate over all organisms. for each organism, use its CDS to calculate the ENC, take the top 5% and calculate CAI weights
organisms_that_caused_error = []

directory = '/Users/tal/Google Drive/TAU/masters/co-translational folding project/Data/codon_usage/complete_CDS/'

for filename in tqdm(os.listdir(directory)):
    path = directory+'/'+filename #the path to the complete CDS of a single organims
    try:
        with open(path) as fasta_file:  
            identifiers = []
            seqs = []
            for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
                identifiers.append(seq_record.id)
                seqs.append((str(seq_record.seq)))

        sequences_df = pd.DataFrame()
        sequences_df['ID'] = identifiers
        sequences_df['sequences'] = seqs #all CDS sequences of a single organism
        
        # clean wrong or short genes

        start_with_ATG = sequences_df['sequences'].apply(lambda x: x[:3] == 'ATG') #keep only sequences that start with ATG
        sequences_df = sequences_df[start_with_ATG]

        too_short = sequences_df['sequences'].apply(lambda x: len(x) < 450) # get sequences that are too short. ENC is not a good measure for shorter sequences.
        sequences_df = sequences_df[~too_short] #remove short sequences

        sequences_df['remainder_from_three'] = sequences_df['sequences'].apply(lambda x: len(x) % 3) #get sequences that do not devide by three (mistakes)
        sequences_df = sequences_df[sequences_df['remainder_from_three'] == 0] # remove mistakes

        not_ambigious = sequences_df['sequences'].apply(lambda x: find_ambigious_alphabet(x)) #keep only sequences with non-ambigious alphabet
        sequences_df = sequences_df[not_ambigious]
        
        #calculate ENC for the remaining CDSs of this organism
        pandarallel.initialize()
        ENC_per_gene = sequences_df['sequences'].parallel_apply(calc_ENC)
        
        #take only CDSs with ENC >= threshold (assumed to be highly expressed)
        threshold = ENC_per_gene.quantile(.95) #top 5%
        highly_biased_genes = ENC_per_gene[ENC_per_gene >= threshold].index 
        seqs = sequences_df.loc[highly_biased_genes]['sequences'].to_list()
        
        #calculate the organism's CAI weights according to the highly expressed genes
        weights = relative_adaptiveness(sequences=seqs)
        
        organism_name = filename.split(".")[0]
        organism_name = organism_name.lower()
        with open('../Results/CAI_tables/'+organism_name+'.pickle', 'wb') as handle:
            pickle.dump(weights, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    except:
        organisms_that_caused_error.append(filename)
        
with open('../Results/organisms_that_caused_error.pickle', 'wb') as handle:
    pickle.dump(organisms_that_caused_error, handle, protocol=pickle.HIGHEST_PROTOCOL)


         

  0%|          | 0/266 [00:00<?, ?it/s]

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


  0%|          | 1/266 [01:51<8:10:49, 111.13s/it]

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


  1%|          | 2/266 [03:14<7:32:56, 102.94s/it]

...

 99%|█████████▉| 264/266 [10:54:25<08:23, 251.62s/it]

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


100%|█████████▉| 265/266 [10:59:53<04:34, 274.28s/it]

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


100%|██████████| 266/266 [11:02:11<00:00, 149.37s/it]


In [18]:
import os
path = '../Results/CAI_tables/'
for filename in os.listdir(path):
    if 'pickle'  in filename:
        temp = filename.split(".")
        os.rename(os.path.join(path,filename), os.path.join(path,temp[0] + '.' + temp[-1]))
